## Veckodagar, avlidna

Jag ska undersöka om det finns något att upptäcka om man grupperar dödsfallen per veckodag.

Jag börjar med att läsa in bladet "Antal avlidna per dag" per dag och plocka bort sista raden 
som är okända. Kolumnen "Datum" lästes inte in som ett datum så jag fick köra en .to_datetime().

In [23]:
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Antal avlidna per dag", names = {"Datum", "Avlidna"}).iloc[0:-1]

df["Datum"] = pd.to_datetime(df["Datum"])

df.tail()

,Datum,Avlidna
585,2021-10-17,2
586,2021-10-18,1
587,2021-10-19,1
588,2021-10-20,1
589,2021-10-21,0


Först konverterar jag datumen till numeriska veckodagar och gör en ny dataframe av den vecko-grupperade och summerade ursprungliga dataframen.

In [24]:
from datetime import datetime

df["Veckodag"] = df["Datum"].dt.dayofweek

df_per_veckodag = df.groupby("Veckodag")[["Avlidna"]].sum().reset_index()

df_per_veckodag

,Veckodag,Avlidna
0,0,2142
1,1,2055
2,2,2234
3,3,2273
4,4,2153
5,5,2022
6,6,2077


In [25]:
import plotly_express as px

# Så här kan vi göra eftersom dataframen är sorterad efter de ursprungliga numeriska veckodagarna
df_per_veckodag["Veckodag"] = ["Måndag", "Tisdag", "Onsdag", "Torsdag", "Fredag", "Lördag", "Söndag"]

fig_px = px.bar(df_per_veckodag, x = "Veckodag", y = "Avlidna", title = "Avlidna per veckodag")

fig_px.write_html(f"./Visualiseringar/avlidna_per_veckodag.html")
fig_px.show()


I resultet ser man inte mycket intressant tycker jag. Skillnaderna är för små för att dra några 
slutsatser. De skillnader som ändå finns kanske kan förklaras med varierande arbetsbelastning för 
den som registerar dödsfallen kombinerat med en osäkerhet när personen avled. Detta är givetvis 
bara gissningar.

## Antal fall regioner över tid

Nu tänkte jag att man kunde titta på hur det såg ut mellan regionerna över tid.
Excellbladet "Antal per dag region" ser intressant ut.

In [26]:
# Antal per dag / region, plotta skillnaderna
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Antal per dag region")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Statistikdatum       626 non-null    datetime64[ns]
 1   Totalt_antal_fall    626 non-null    int64         
 2   Blekinge             626 non-null    int64         
 3   Dalarna              626 non-null    int64         
 4   Gotland              626 non-null    int64         
 5   Gävleborg            626 non-null    int64         
 6   Halland              626 non-null    int64         
 7   Jämtland_Härjedalen  626 non-null    int64         
 8   Jönköping            626 non-null    int64         
 9   Kalmar               626 non-null    int64         
 10  Kronoberg            626 non-null    int64         
 11  Norrbotten           626 non-null    int64         
 12  Skåne                626 non-null    int64         
 13  Stockholm            626 non-null  

Okej, det ser ju fint ut...

In [28]:
df_antal_dag_region = df.copy()

# Skapar en lista över regionerna från columnerna för att ha till y-axeln när jag plottar
columns = [column for column in df_antal_dag_region.columns][2:]

# Skapar ett rullande medelvärde över 7 dagar och droppar de rader i början som inte fått några värden
df_antal_dag_region[columns] = df_antal_dag_region[columns].rolling(7).mean().dropna()

fig_px = px.line(df_antal_dag_region, x = "Statistikdatum", y = columns)
fig_px.write_html(f"./Visualiseringar/antal_per_dag_region.html")
fig_px.show()

Statistiken var väldigt hackig utan medelvärdet, troligtvis beror det på att ingen statistik fördes in på helgerna.

Med det rullande medelvärdet ser det bättre ut, men det är ganska rörigt och något speciellt framkommer inte. Det verkar
dock som att Stockholm nästan alltid går in i "vågorna" ett par veckor tidigare än övriga regioner.

## Antal fall per capita och region

För att justera bort den stora skillnaden i föregående "analys" ska jag undersöka hur det ser ut 
vecka för vecka per 100 000 invåndare i regionerna.

Blad "Veckodata Region", således...

In [29]:
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Veckodata Region")

df["Vecka"] = df["år"].astype(str) + "v" + df["veckonummer"].astype(str)

df.head()

,år,veckonummer,Region,Antal_fall_vecka,Kum_antal_fall,Antal_intensivvårdade_vecka,Kum_antal_intensivvårdade,Antal_avlidna_vecka,Kum_antal_avlidna,Antal_fall_100000inv_vecka,Kum_fall_100000inv,Vecka
0,2020,1,Blekinge,0,0,0,0,0,0,0,0,2020v1
1,2020,2,Blekinge,0,0,0,0,0,0,0,0,2020v2
2,2020,3,Blekinge,0,0,0,0,0,0,0,0,2020v3
3,2020,4,Blekinge,0,0,0,0,0,0,0,0,2020v4
4,2020,5,Blekinge,0,0,0,0,0,0,0,0,2020v5


In [30]:
df_veckodata_region = df[["Vecka", "Region", "Antal_fall_100000inv_vecka"]]

fig_px = px.line(df_veckodata_region, x = "Vecka", y = "Antal_fall_100000inv_vecka", color = "Region")
fig_px.write_html(f"./Visualiseringar/antal_per100000_vecka_region.html")
fig_px.show()

När man justerar för folkmängd utmärker sig inte längre Stockholm. Det är svårt att säga att någon 
region är sämre än någon annan. En intressant detalj dock är Gotlands extrema topp 2021v31, märkligt
nog bara en-två veckor efter att Stockholmsveckan  